## AdaBoost Hyperparameter Tuning (Recall)

In [31]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score

from sklearn.ensemble import AdaBoostClassifier

import optuna

In [33]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [35]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [37]:
# Hyperparameter tuning

def objective(trial):

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0)
    }

    model = AdaBoostClassifier(
        **params,
        random_state=42)
    
    threshold = 0.5
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(recall_score(y_te, y_pred))
        
    return np.mean(scores)

In [39]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-08 21:49:48,910] A new study created in memory with name: no-name-dcdf618b-333b-4ca1-b77b-e9d1b66ac665


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-08 21:50:09,386] Trial 0 finished with value: 0.4484662576687117 and parameters: {'n_estimators': 953, 'learning_rate': 0.8618864425510021}. Best is trial 0 with value: 0.4484662576687117.
[I 2024-07-08 21:50:18,664] Trial 1 finished with value: 0.44907975460122695 and parameters: {'n_estimators': 443, 'learning_rate': 0.6682053805700706}. Best is trial 1 with value: 0.44907975460122695.
[I 2024-07-08 21:50:32,704] Trial 2 finished with value: 0.4441717791411043 and parameters: {'n_estimators': 686, 'learning_rate': 0.11106321149959954}. Best is trial 1 with value: 0.44907975460122695.
[I 2024-07-08 21:50:40,388] Trial 3 finished with value: 0.4404907975460123 and parameters: {'n_estimators': 371, 'learning_rate': 0.1652324979178452}. Best is trial 1 with value: 0.44907975460122695.
[I 2024-07-08 21:50:49,201] Trial 4 finished with value: 0.4466257668711656 and parameters: {'n_estimators': 406, 'learning_rate': 0.3250376281030502}. Best is trial 1 with value: 0.4490797546012

In [43]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=88, state=1, values=[0.44969325153374234], datetime_start=datetime.datetime(2024, 7, 8, 22, 8, 4, 674661), datetime_complete=datetime.datetime(2024, 7, 8, 22, 8, 7, 961835), params={'n_estimators': 159, 'learning_rate': 0.7814626294518465}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=50, step=1), 'learning_rate': FloatDistribution(high=1.0, log=False, low=0.001, step=None)}, trial_id=88, value=None)
Best hyperparameters: {'n_estimators': 159, 'learning_rate': 0.7814626294518465}
